In [137]:
import sys

def EulerianCycle(edges) :
    path = []
    edge_check = []
    edge_count = []
    
    node_count = 0
    edge_total_count = 0
    
    node_list = []
    
    for edge in edges :
        edge = edge.split()
        start_node = int(edge[0])
        end_node = list(map(int, edge[2].split(',')))
        
        if(start_node not in node_list) : node_list.append(start_node)
        for i in range(0, len(end_node)) :
            if(end_node[i] not in node_list) : node_list.append(end_node[i])
        
    node_count = len(node_list)
    for i in range(0, node_count + 1) :
        path.append([])
        edge_check.append([])
        edge_count.append([])
    
    for edge in edges :
        edge = edge.split()
        start_node = int(edge[0])
        end_node = list(map(int, edge[2].split(',')))
        
        path[start_node] = end_node
        edge_count[start_node] = len(end_node)
        edge_total_count += len(end_node)
        
    for i in range(0, node_count) :
        for j in range(0, len(path[i])) : edge_check[i].append(False)
            
    present_node = node_list[0]
    cycle = [present_node]
    
    while edge_count[present_node] != 0 :
        for i in range(0, len(path[present_node])) :
            if(edge_check[present_node][i] == False) :
                edge_check[present_node][i] = True; edge_count[present_node] -= 1; edge_total_count -= 1
                present_node = path[present_node][i]
                cycle.append(present_node)
                break
                
    while edge_total_count != 0 :
        loop_out = False
        for i in range(0, len(cycle)) :
            for j in range(0, len(path[cycle[i]])) :
                if(edge_check[cycle[i]][j] == False) :
                    present_node = cycle[i]
                    cycle = cycle[i:] + cycle[1:i + 1]
                    loop_out = True
                    break
            if(loop_out == True) : break
        
        while edge_count[present_node] != 0 :
            for i in range(0, len(path[present_node])) :
                if(edge_check[present_node][i] == False) :
                    edge_check[present_node][i] = True; edge_count[present_node] -= 1; edge_total_count -= 1
                    present_node = path[present_node][i]
                    cycle.append(present_node)
                    break
                    
    return cycle

#whole_edges = sys.stdin.read().splitlines()
input_file = open("untitled.txt", 'r')
whole_edges = input_file.readlines()
Euler_Path = EulerianCycle(whole_edges)

printline = "%s" % Euler_Path[0]
for i in range(1, len(Euler_Path)) :
    printline += "->%s" % Euler_Path[i]
print(printline)

2022->2020->1221->1220->1219->744->742->337->338->1112->1111->1113->338->2933->2932->2934->338->339->126->125->124->2280->2279->2278->124->120->778->1455->1454->1453->2723->2722->2724->1453->778->780->779->1145->1146->1144->1303->1359->1357->2241->2240->2554->2555->2556->2240->2983->2984->2985->2240->2239->1357->1358->1303->1595->1599->1598->1597->1595->1594->1596->1303->1304->1843->1844->2199->2198->2197->1844->1845->1304->1305->2037->2035->2036->1305->1144->779->1171->2075->2074->2981->2982->2980->2074->2076->2258->2257->2259->2076->2583->2582->2581->2076->1171->1173->1172->779->120->118->80->67->198->1805->1806->1804->198->196->220->842->843->1057->1058->1059->2093->2092->2094->1059->843->841->220->222->221->232->234->2298->2296->2297->234->233->221->419->420->418->1943->2838->2836->2837->1943->1944->1942->418->221->196->197->67->69->1466->2371->2373->2372->1466->2385->2384->2383->1466->2772->2771->2770->1466->1467->1465->69->68->1293->2621->2620->2622->1293->1292->1291->68->1737->1

In [210]:
import sys
from functools import reduce

def EulerianSingleCycleDFS(edges, current_node) :
    stack = []
    circuit = []

    while True :
        while(bool(edges[current_node])) :
            stack.append(current_node) # add nodes to visit
            current_node = edges[current_node].pop() # erase the edge
            
        circuit.append(current_node) # add current node
        if(stack != []) : current_node = stack.pop() # determine next node from stack if stack is not empty
        else : return circuit[::-1], edges # reverse the circuit -> returning DFS path

def BalancingEdges(edges) :
    out_nodes = list(edges.keys())
    in_nodes = reduce(lambda x, y : x + y, edges.values())
    whole_nodes = set(in_nodes + out_nodes)
    
    unbalanced_to = -1; unbalanced_from = -1
    
    for node in whole_nodes :
        in_count = in_nodes.count(node)
        out_count = 0
        if(node in out_nodes) : out_count = len(edges[node])
        else : edges[node] = []
            
        if(in_count > out_count) : unbalanced_from = node # in > out    ->    starting node
        elif(out_count > in_count) : unbalanced_to = node # out > in    ->    ending node
    
    if(unbalanced_from in out_nodes) : edges[unbalanced_from].append(unbalanced_to) # make fake edges, balancing edges table
    elif(unbalanced_from != -1) : edges[unbalanced_from] = [unbalanced_to]
        
    return edges, unbalanced_from, unbalanced_to
        
def EulerianPath(edges):
    #choose current location and initialize stack and circuit
    edges, ubf, ubt = BalancingEdges(edges)
    circuit, edges = EulerianSingleCycleDFS(edges, list(edges.keys())[0]) # first circle
    while True : # make circles until there is no edge
        iteration = 0
        for path_node in circuit :
            if(bool(edges[path_node])) :
                added_circuit, edges = EulerianSingleCycleDFS(edges, path_node) # additional circle
                circuit = circuit[:iteration] + added_circuit + circuit[iteration + 1:] # interconnect the previous circle and the new one
                break
            iteration += 1
        if(iteration == len(circuit)) : break
            
    division_point = list(filter(lambda i : circuit[i:i+2] == [ubf, ubt], range(len(circuit) - 1))) # find fake edge
    if(bool(division_point) == False) : return circuit # if there is no fake edge(already fully balanced edges), we don't need division
    return circuit[division_point[0] + 1:] + circuit[1:division_point[0] + 1]

lines = sys.stdin.read().splitlines()
#input_file = open("untitled.txt", 'r')
#lines = input_file.read().splitlines()

edges = {}
for line in lines:
    paths = line.rstrip().split(" -> ")
    edges[paths[0]] = paths[1].split(",")
    
circuit = EulerianPath(edges)
print("->".join(circuit))

32->31->915->913->914->1322->1321->1323->1446->2283->2281->2738->2739->2737->2281->2282->1446->1445->1864->1865->1866->1445->1444->1323->914->31->4->2162->2178->2176->2177->2162->2163->2161->4->29->62->63->61->92->93->569->2200->2202->2201->569->568->2685->2683->2684->568->570->2469->2468->2467->570->93->438->437->436->93->91->61->752->751->2338->2339->2340->751->753->61->29->34->454->2137->2139->2138->454->456->455->940->2474->2473->2475->940->2326->2328->2327->940->942->2043->2041->2042->942->941->974->975->1839->1838->1951->1952->1953->1838->1837->975->973->941->455->34->35->319->320->321->1612->1613->1614->321->35->36->29->30->987->986->1163->1162->1164->986->985->30->64->2303->2302->2331->2330->2329->2302->2304->64->65->68->2047->2048->2049->68->1138->2751->2749->2750->1138->1139->1140->68->69->177->175->646->648->967->968->969->648->647->175->374->1942->1944->2117->2116->2118->1944->1943->374->375->2229->2228->2227->375->1603->1605->1604->375->373->825->824->2565->2564->2563->824

In [227]:
import sys
from itertools import product
from functools import reduce

def EulerianSingleCycleDFS(edges, current_node) :
    stack = []
    circuit = []

    while True :
        while(bool(edges[current_node])) :
            stack.append(current_node) # add nodes to visit
            current_node = edges[current_node].pop() # erase the edge
            
        circuit.append(current_node) # add current node
        if(stack != []) : current_node = stack.pop() # determine next node from stack if stack is not empty
        else : return circuit[::-1], edges # reverse the circuit -> returning DFS path

def BalancingEdges(edges) :
    out_nodes = list(edges.keys())
    in_nodes = reduce(lambda x, y : x + y, edges.values())
    whole_nodes = set(in_nodes + out_nodes)
    
    unbalanced_to = -1; unbalanced_from = -1
    
    for node in whole_nodes :
        in_count = in_nodes.count(node)
        out_count = 0
        if(node in out_nodes) : out_count = len(edges[node])
        else : edges[node] = []
            
        if(in_count > out_count) : unbalanced_from = node # in > out    ->    starting node
        elif(out_count > in_count) : unbalanced_to = node # out > in    ->    ending node
    
    if(unbalanced_from in out_nodes) : edges[unbalanced_from].append(unbalanced_to) # make fake edges, balancing edges table
    elif(unbalanced_from != -1) : edges[unbalanced_from] = [unbalanced_to]
        
    return edges, unbalanced_from, unbalanced_to
        
def EulerianPath(edges):
    #choose current location and initialize stack and circuit
    edges, ubf, ubt = BalancingEdges(edges)
    circuit, edges = EulerianSingleCycleDFS(edges, list(edges.keys())[0]) # first circle
    
    while True : # make circles until there is no edge
        iteration = 0
        for path_node in circuit :
            if(bool(edges[path_node])) :
                added_circuit, edges = EulerianSingleCycleDFS(edges, path_node) # additional circle
                circuit = circuit[:iteration] + added_circuit + circuit[iteration + 1:] # interconnect the previous circle and the new one
                break
            iteration += 1
        if(iteration == len(circuit)) : break
            
    division_point = list(filter(lambda i : circuit[i:i+2] == [ubf, ubt], range(len(circuit) - 1))) # find fake edge
    if(bool(division_point) == False) : return circuit # if there is no fake edge(already fully balanced edges), we don't need division
    return circuit[division_point[0] + 1:] + circuit[1:division_point[0] + 1]

def universal_string(k):
    universal_dict = {}
    for kmer in [''.join(item) for item in product("01", repeat = k)] : # Make Cartersian Product
        if(kmer[:-1] in universal_dict) : universal_dict[kmer[:-1]].append(kmer[1:]) # Make edge
        else : universal_dict[kmer[:-1]] = [kmer[1:]]
            
    path = EulerianPath(universal_dict) # Make EulerPath
    printline = path[0]
    for i in range(1, 2 ** k - (k - 2)) : # Joining nodes
        printline += path[i][k - 2]
    print(printline)
    
lines = sys.stdin.read().splitlines()
#input_file = open("untitled.txt", 'r')
#lines = input_file.read().splitlines()
k = int(lines[0])

universal_string(k)

00000101111111110111111010111110110111110011111110010111101110111101010111100110111100011111100010111011010111010110111010011111010010111001110111001010111000110111000011111000010110110110011110110010110101010110100110110100011110100010110011010110010011110010010110001110110001010110000110110000011110000010101010011101010010101000110101000011101000010100110011100110010100100110100100011100100010100010011100010010100000110100000011100000010011000110011000010010010000110010000010001100010001000010000000110000


In [229]:
from functools import reduce
import sys

#input_file = open("untitled.txt", 'r')
#kmers = input_file.read().splitlines()

kmers = [line.strip() for line in sys.stdin.read().splitlines()]
edges = {}

for kmer in kmers:
    if kmer[:-1] in edges : edges[kmer[:-1]].append(kmer[1:])
    else : edges[kmer[:-1]] = [kmer[1:]]

balanced, unbalanced = [], []
out_values = reduce(lambda a, b : a + b, edges.values())

for node in set(out_values + list(edges.keys())):
    out_value = out_values.count(node)
    if node in edges : in_value = len(edges[node])
    else : in_value = 0
    if in_value == out_value == 1 : balanced.append(node)
    else : unbalanced.append(node)

import collections
def flatten(x):
    result = []
    for el in x:
        if isinstance(x, collections.Iterable) and not isinstance(el, str):
            result.extend(flatten(el))
        else:
            result.append(el)
    return result

get_contigs = lambda s, c : flatten([c + edge[-1] if edge not in balanced else get_contigs(edge, c + edge[-1]) for edge in edges[s]])
contigs = sorted(flatten([get_contigs(start,start) for start in set(unbalanced) & set(edges.keys())]))

for contig in contigs : print(contig)

... 


In [3]:
import sys

def make_edges(seqs):

    edges = []
    for read in seqs:
        edges.append(read[:-1]+ ' -> ' +read[1:])
    return edges

def reconstruct_string(edges):

    string_dict = {line.strip().split(' -> ')[0]:line.strip().split(' -> ')[1] for line in edges}

    head = filter(lambda x : x not in string_dict.values(), string_dict.keys())[0]
    tail = filter(lambda x : x not in string_dict.keys(), string_dict.values())[0]

    reconstructed_str = head[0]
    current_str = head

    while current_str != tail:
        current_str = string_dict[current_str]
        reconstructed_str += current_str[0]
    reconstructed_str += tail[1:]

    return reconstructed_str

input_file = open("untitled.txt", 'r')
lines = input_file.read().splitlines()

k = int(lines[0])
reads = lines[1:]

edges = make_edges(reads)
ans = reconstruct_string(edges)
print ans

GCCTAGTGAACTGGAGTGCCAGAGGTGACAAGAGCAAGGCAGAGCGATATCTGTGTTTGGTAGGAACGCAACGAATAACTGAATCGTACCTATTGATAGAGACGGAGTACATATGAACTTGCGTCACGTGGTACACCGATATCCCGGACGTATCCTTCGCTATAATGACGCTAAGCCGTATAATCTTTACGTCACCTTGGAGAATGACGGTCTTTGGATCAGAGCACGGTCTCCTCCCGTACAGTACACCCGATAGTCGGAAACGTAGTATAAAGGAGTCTTAACGAGGCAAGTTTGGCGTCCCACTCCAGAATACATATAAACGTTGAGCGGCTCTCCGCGAATTCACTAGACAGTAACGGTCCCACATCAAAGATCACTGGAGGGCAGTTCACAGGCCACCAGCACTATGTGCTAAAGAATTATGCCGGTCCCCGCATGTCAGGTGTCTATGGCAGCTTGGTGTTAACTTGGGCGTCTAGAATTCTTCAGGCTTTGTTTGTGCGTGTATGTTTTGCGCGAGCGTCGCCACTAATGCTTAATGGAGCAGTCCAACCACTAGGATTTGAATCTTGCCGTCTAGCCAGTAGAGCTCACCCGTATGCGATCATAGACCAAAGCTATTTCTAACTATAAACCTAAGCTATTACGGCCGGTGCGCTCCTCACTCGCTTTGCTTTCTCAAAGCAGCGGGAGTACGTTCACCCCTTGCTAGGCTCATCCGGAATGCTAATGACCTGTTGTGGCGTGCTTACTTGGTAACGGGCGAGACGTAACGCCATCGTGAATGGAGAAGAGTTGAAAATTCAATTGGCCAATATGTGTGACTACGGGGAGGCTGCTGGTACGGTCAATCGATGGCTGCCACTCTGCAACAGCTGAGGCCGTGGGGCCAAAACGCAGTGTTGCTAAACGGGTATTATTGACATACTAGTCCCTTTGCGATGTATTATCACCTCGGCTCTGGCCTAAGCATAGTACCCCGTATCTAAAGTATAAC